In [29]:
import pickle
import logging 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)
from sklearn.inspection import permutation_importance
from sklearn.utils import resample

# Corrected logging setup
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [30]:
import os

file_path = 'data/raw/qsar_aquatic_toxicity.csv'
if os.path.exists(file_path):
    print("El archivo existe.")
else:
    print("El archivo no existe en la ruta especificada.")

El archivo no existe en la ruta especificada.


In [31]:
df_raw = pd.read_csv(workspaces/4geeks_class/data/raw/qsar_aquatic_toxicity.csv, sep=',', header=None)

SyntaxError: invalid decimal literal (2547800281.py, line 1)


### **3. Exploratory Data Analysis**
**Objective**: Summarize the main characteristics of the dataset using descriptive statistics and data visualization methods

In [ ]:
df_train, df_test = train_test_split(
    df, test_size=0.15,
    random_state=2025
)
df_train, df_val = train_test_split(
    df_train, test_size=0.15
    random_state=2025
)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_train.info()


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1541689914.py, line 6)

In [ ]:
df_train.describe().T

In [ ]:
df_train.hist()
plt.tight_layout()
plt.show

In [ ]:
sns.pairplot(df_train, diag_kind='kde', corner=True)
plt.show()

In [ ]:
sns.heatmap(df_train.corr(), vmin=-1, vmax=1, cmap='RdBu', annot=True)
